In [5]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext
import json
from pyspark import SparkConf

def stopword_removal(word):
    if word not in stop_words_set:
        return ''.join(ch for ch in word if ch not in remove)


import re
if __name__ == '__main__':
    
    read_file = sys.argv[1]
    write_file = sys.argv[2]
    read_stop = sys.argv[3]
    target_year = sys.argv[4]
    m_user = sys.argv[5]
    n_word = sys.argv[6]
    remove = set(r"""()[],.!?:;""")
    stop_words_set = set(word.strip() for word in open(read_stop))
    sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
    ans = dict()
    input_lines = sc.textFile(read_file).map(lambda row: json.loads(row))
    
    data = input_lines.map(lambda kv: (kv['review_id'] ,kv['date'],kv['user_id'],kv['business_id'],kv['text']))
    
    ans['A'] =data.count()
    
    ans['B'] = data.map(lambda kv: kv[1]).map(lambda kv : kv[0]+kv[1]+kv[2]+kv[3] ) \
              .filter(lambda kv:int(kv) == target_year).count()

    ans['C'] = data.map(lambda kv : kv[2]).distinct().count()
    
    ans['D'] = data.map(lambda kv: (kv[2], 1)).reduceByKey(lambda a, b: a + b) \
              .takeOrdered(m_user, key=lambda kv: (-kv[1], kv[0]))
     
    words_dict = data.map(lambda kv: kv[4]).flatMap(lambda text: text.lower().split(' ')) \
                 .map(lambda word: (stopword_removal(word), 1)) \
                 .filter(lambda kv: (kv[0] is not None) and (kv[0] is not "")) \
                .reduceByKey(lambda a, b: a + b).takeOrdered(n_word, key=lambda kv: (-kv[1], kv[0]))
    ans['E'] =list(map(lambda kv: kv[0], words_dict))
    json.dump(ans, open("output.json","w"))
